Needed imports and utility

In [ ]:
import os
import ansys.speos.script as script
import ansys.speos.core as core

tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")

def clean_all_dbs(speos_client: core.SpeosClient):
    for item in (
        speos_client.jobs().list()
        + speos_client.scenes().list()
        + speos_client.simulation_templates().list()
        + speos_client.sensor_templates().list()
        + speos_client.source_templates().list()
        + speos_client.intensity_templates().list()
        + speos_client.spectrums().list()
        + speos_client.vop_templates().list()
        + speos_client.sop_templates().list()
        + speos_client.parts().list()
        + speos_client.bodies().list()
        + speos_client.faces().list()
    ):
        item.delete()

tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")

Create connection with speos rpc server

In [ ]:
speos = core.Speos(host="localhost", port=50051)
clean_all_dbs(speos.client)

New project

In [ ]:
p = script.Project(speos=speos)
print(p)

# Create
Create locally  
The mention "local: " is added when printing the spectrum  
For sources, it is always important to set properties.

In [ ]:
intensity_file_path = os.path.join(tests_data_path, "IES_C_DETECTOR.ies")

source1 = p.create_source(name="Source.1")
source1.set_luminaire().set_intensity_file_uri(uri=intensity_file_path)
source1.set_luminaire_properties()
print(source1)

Push it to the server.  
Now that it is committed to the server, the mention "local: " is no more present when printing the source.  
Except for the properties that stay local until the whole project is committed.

In [ ]:
source1.commit()
print(source1)

Another example by setting several characteristics

In [ ]:
intensity_file_path = os.path.join(tests_data_path, "IES_C_DETECTOR.ies")

source2 = p.create_source(name="Source.2")
source2.set_luminaire().set_intensity_file_uri(uri=intensity_file_path)\
                       .set_flux_radiant()\
                       .set_spectrum().set_blackbody()
source2.set_luminaire_properties([20, 50, 10, 1, 0, 0, 0, 1, 0, 0, 0, 1])\
       .commit()
print(source2)

## Default values
Some default values are available when applicable in every methods and class.

# Read
## Source Instance
A mention "local: " is added if it is not yet committed to the server

In [ ]:
print(source1)

## Project

In [ ]:
print(p)

# Update
Tipp: if you are manipulating a source already committed, don't forget to commit your changes.  
If you don't, you will still only watch what is committed on the server.

In [ ]:
source1.set_luminaire().set_flux_radiant(value=1.2)\
                       .set_spectrum().set_halogen()
source1.set_luminaire_properties([17, 10, 10, 1, 0, 0, 0, 1, 0, 0, 0, 1])\
       .commit()
print(source1)

Be careful to have something coherent when updating.  
Example just after source of type luminaire but with rayfile properties.

In [ ]:
source1.set_rayfile_properties()

## Reset
Possibility to reset local values from the one available in the server.

In [ ]:
source1.set_rayfile_properties() # set rayfile_properties but no commit
source1.reset() # reset -> this will apply the server value to the local value
source1.delete() # delete (to display the local value with the below print)
print(source1)

# Delete
Once the data is deleted from the server, you can still work with local data and maybe commit later.

In [ ]:
source2.delete()
print(source2)

In [ ]:
source1.delete()

# More content
## ray-file source

In [ ]:
ray_file_path = os.path.join(tests_data_path, "Rays.ray")

source3 = p.create_source(name="Source.3")
source3.set_rayfile().set_ray_file_uri(uri=ray_file_path)
source3.set_rayfile_properties().commit()
print(source3)

In [ ]:
source3.set_rayfile().set_flux_luminous()
source3.commit()
print(source3)

In [ ]:
source3.delete()

## surface source

In [ ]:
source4 = p.create_source(name="Source.4")
source4.set_surface().set_exitance_constant()
source4.set_surface_properties().set_exitance_constant_properties([(script.GeoRef.from_native_link("TheBodyB/TheFaceF"), False), 
                                                                   (script.GeoRef.from_native_link("TheBodyB/TheFaceG"), True)])
source4.commit()
print(source4)

In [ ]:
source4.set_surface().set_flux_luminous_intensity()
source4.set_surface().set_intensity().set_gaussian().set_gaussian_properties([10,50,20,1,0,0,0,1,0,0,0,1])
source4.commit()
print(source4)

In [ ]:
source4.delete()
print(source4)

# Cleanup

In [ ]:
clean_all_dbs(speos.client)